# Test NIP-66 Relay Monitoring

Interactive notebook for testing the `Nip66` model:
- **RTT** - Round-trip time tests (open, read, write)
- **SSL** - Certificate checks (clearnet only)
- **GEO** - Geolocation lookup (clearnet only)
- **DNS** - DNS resolution (clearnet only)
- **HTTP** - HTTP headers (clearnet or via proxy)
- **Proxy support** - For Tor/I2P/Loki relays (RTT and HTTP tests only)

**RTT test requires:** `keys`, `event_builder`, `read_filter`

**GeoIP databases path:** `implementations/bigbrotr/static/`

In [1]:
import sys
sys.path.insert(0, "../src")

import asyncio
import json
import logging
from pathlib import Path

import geoip2.database
from nostr_sdk import EventBuilder, Filter

from models.nip66 import Nip66, Nip66TestError
from models.relay import Relay
from models.keys import Keys
from models.metadata import Metadata

# Enable logging for nip66 module
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s %(levelname)-5s %(name)s | %(message)s",
    datefmt="%H:%M:%S",
)
logging.getLogger("nip66").setLevel(logging.DEBUG)
logging.getLogger("nostr_sdk").setLevel(logging.WARNING)

print("Imports loaded successfully")

Imports loaded successfully


## 1. Setup GeoIP Databases and Keys

In [2]:
# Load GeoIP databases
geoip_dir = Path("../implementations/bigbrotr/static")

city_db_path = geoip_dir / "GeoLite2-City.mmdb"
asn_db_path = geoip_dir / "GeoLite2-ASN.mmdb"

print(f"City DB exists: {city_db_path.exists()}")
print(f"ASN DB exists: {asn_db_path.exists()}")

city_reader = geoip2.database.Reader(str(city_db_path)) if city_db_path.exists() else None
asn_reader = geoip2.database.Reader(str(asn_db_path)) if asn_db_path.exists() else None

print(f"\nCity reader: {city_reader is not None}")
print(f"ASN reader: {asn_reader is not None}")

City DB exists: True
ASN DB exists: True

City reader: True
ASN reader: True


In [3]:
# Generate test keys
keys = Keys.generate()
print(f"Test pubkey: {keys.public_key().to_hex()[:16]}...")

# Create event builder for write tests
event_builder = EventBuilder.text_note("NIP-66 test event")
print(f"Event builder ready")

# Create read filter for read tests
read_filter = Filter().limit(1)
print(f"Read filter ready")

Test pubkey: f06640b328900700...
Event builder ready
Read filter ready


## 2. Full Test on a Clearnet Relay

In [4]:
relay = Relay("wss://relay.damus.io")
print(f"Testing: {relay.url}")
print(f"Network: {relay.network}")

Testing: wss://relay.damus.io
Network: clearnet


In [5]:
# Run full NIP-66 test (all 5 metadata types)
nip66 = await Nip66.test(
    relay,
    keys=keys,
    event_builder=event_builder,
    read_filter=read_filter,
    city_reader=city_reader,
    asn_reader=asn_reader,
)

print(f"\nMetadata collected:")
print(f"  RTT:  {nip66.rtt_metadata is not None}")
print(f"  SSL:  {nip66.ssl_metadata is not None}")
print(f"  GEO:  {nip66.geo_metadata is not None}")
print(f"  DNS:  {nip66.dns_metadata is not None}")
print(f"  HTTP: {nip66.http_metadata is not None}")

22:00:29 INFO  models.nip66 | test: relay=wss://relay.damus.io timeout=10.0s
22:00:29 DEBUG models.nip66 | test: running tests=['dns', 'rtt', 'ssl', 'geo', 'http']
22:00:29 DEBUG models.nip66 | _test_dns: relay=wss://relay.damus.io timeout=10.0s
22:00:29 DEBUG models.nip66 | _test_dns: resolving host=relay.damus.io
22:00:29 DEBUG models.nip66 | _test_rtt: relay=wss://relay.damus.io timeout=10.0s proxy=None
22:00:29 DEBUG models.nip66 | _test_ssl: relay=wss://relay.damus.io timeout=10.0s
22:00:29 DEBUG models.nip66 | _test_ssl: checking host=relay.damus.io port=443
22:00:29 DEBUG models.nip66 | _test_geo: relay=wss://relay.damus.io city=True asn=True
22:00:29 DEBUG models.nip66 | _test_geo: resolving host=relay.damus.io
22:00:29 DEBUG models.nip66 | _test_http: relay=wss://relay.damus.io timeout=10.0s proxy=None
22:00:29 DEBUG models.nip66 | _test_rtt: connecting relay=wss://relay.damus.io
22:00:29 DEBUG models.nip66 | _test_geo: resolved ip=172.67.206.128 relay=wss://relay.damus.io
22:


Metadata collected:
  RTT:  True
  SSL:  True
  GEO:  True
  DNS:  True
  HTTP: True


## 3. RTT Metadata

In [6]:
if nip66.rtt_metadata:
    rtt = nip66.rtt_metadata.data
    print("=== RTT Results ===")
    print(f"rtt_open:  {rtt['rtt_open']} ms")
    print(f"rtt_read:  {rtt['rtt_read']} ms")
    print(f"rtt_write: {rtt['rtt_write']} ms")
else:
    print("No RTT data collected")

=== RTT Results ===
rtt_open:  338 ms
rtt_read:  206 ms
rtt_write: None ms


## 4. SSL Metadata

In [7]:
if nip66.ssl_metadata:
    ssl = nip66.ssl_metadata.data
    print("=== SSL Results ===")
    print(f"ssl_valid:      {ssl['ssl_valid']}")
    print(f"ssl_subject_cn: {ssl['ssl_subject_cn']}")
    print(f"ssl_issuer:     {ssl['ssl_issuer']}")
    print(f"ssl_protocol:   {ssl['ssl_protocol']}")
    print(f"ssl_cipher:     {ssl['ssl_cipher']}")
else:
    print("No SSL data (overlay relay or ws://)")

=== SSL Results ===
ssl_valid:      True
ssl_subject_cn: None
ssl_issuer:     None
ssl_protocol:   TLSv1.3
ssl_cipher:     TLS_AES_256_GCM_SHA384


## 5. GEO Metadata

In [8]:
if nip66.geo_metadata:
    geo = nip66.geo_metadata.data
    print("=== GEO Results ===")
    print(f"geo_ip:         {geo.get('geo_ip')}")
    print(f"geo_country:    {geo.get('geo_country')}")
    print(f"geo_city:       {geo.get('geo_city')}")
    print(f"geo_asn:        {geo.get('geo_asn')}")
    print(f"geo_asn_org:    {geo.get('geo_asn_org')}")
else:
    print("No GEO data (overlay relay or no GeoIP DB)")

=== GEO Results ===
geo_ip:         172.67.206.128
geo_country:    US
geo_city:       None
geo_asn:        13335
geo_asn_org:    CLOUDFLARENET


## 6. DNS Metadata

In [9]:
if nip66.dns_metadata:
    dns = nip66.dns_metadata.data
    print("=== DNS Results ===")
    print(f"dns_ip:   {dns['dns_ip']}")
    print(f"dns_ipv6: {dns['dns_ipv6']}")
    print(f"dns_ns:   {dns['dns_ns']}")
    print(f"dns_ttl:  {dns['dns_ttl']} seconds")
    print(f"dns_rtt:  {dns['dns_rtt']} ms")
else:
    print("No DNS data (overlay relay)")

=== DNS Results ===
dns_ip:   104.21.61.55
dns_ipv6: 2606:4700:3033::6815:3d37
dns_ns:   ['elsa.ns.cloudflare.com', 'paul.ns.cloudflare.com']
dns_ttl:  123 seconds
dns_rtt:  18 ms


## 7. HTTP Metadata

In [10]:
if nip66.http_metadata:
    http = nip66.http_metadata.data
    print("=== HTTP Results ===")
    print(f"http_server:     {http['http_server']}")
    print(f"http_powered_by: {http['http_powered_by']}")
else:
    print("No HTTP data")

=== HTTP Results ===
http_server:     cloudflare
http_powered_by: None


## 8. Full JSON Output

In [11]:
full_data = {
    "relay_url": nip66.relay.url,
    "generated_at": nip66.generated_at,
    "rtt": nip66.rtt_metadata.data if nip66.rtt_metadata else None,
    "ssl": nip66.ssl_metadata.data if nip66.ssl_metadata else None,
    "geo": nip66.geo_metadata.data if nip66.geo_metadata else None,
    "dns": nip66.dns_metadata.data if nip66.dns_metadata else None,
    "http": nip66.http_metadata.data if nip66.http_metadata else None,
}

print(json.dumps(full_data, indent=2))

{
  "relay_url": "wss://relay.damus.io",
  "generated_at": 1767646839,
  "rtt": {
    "rtt_open": 338,
    "rtt_read": 206,
    "rtt_write": null
  },
  "ssl": {
    "ssl_valid": true,
    "ssl_subject_cn": null,
    "ssl_issuer": null,
    "ssl_issuer_cn": null,
    "ssl_expires": null,
    "ssl_not_before": null,
    "ssl_san": null,
    "ssl_serial": null,
    "ssl_version": null,
    "ssl_fingerprint": "SHA256:52:98:84:18:76:27:20:17:6E:A6:FE:A5:8D:2F:E3:B3:0E:3F:33:ED:E9:69:86:AB:3F:DF:78:26:28:59:DC:38",
    "ssl_protocol": "TLSv1.3",
    "ssl_cipher": "TLS_AES_256_GCM_SHA384",
    "ssl_cipher_bits": 256
  },
  "geo": {
    "geo_ip": "172.67.206.128",
    "geo_country": "US",
    "geo_country_name": "United States",
    "geo_continent": null,
    "geo_continent_name": null,
    "geo_is_eu": false,
    "geo_region": null,
    "geo_city": null,
    "geo_postal": null,
    "geo_lat": null,
    "geo_lon": null,
    "geo_accuracy": null,
    "geo_tz": null,
    "geohash": null,
    "g

## 9. Conversion to RelayMetadata

In [12]:
# Convert to RelayMetadata for DB storage (5 objects)
rtt_meta, ssl_meta, geo_meta, dns_meta, http_meta = nip66.to_relay_metadata()

print("=== RelayMetadata Objects ===")
for name, meta in [("RTT", rtt_meta), ("SSL", ssl_meta), ("GEO", geo_meta), ("DNS", dns_meta), ("HTTP", http_meta)]:
    if meta:
        print(f"{name}: {meta.metadata_type}")
    else:
        print(f"{name}: None")

=== RelayMetadata Objects ===
RTT: nip66_rtt
SSL: nip66_ssl
GEO: nip66_geo
DNS: nip66_dns
HTTP: nip66_http


## 10. Test Multiple Relays

In [13]:
relay_urls = [
    "wss://relay.damus.io",
    "wss://nos.lol",
    "wss://relay.nostr.band",
    "wss://nostr.wine",
]

async def test_nip66_safe(url: str) -> tuple[str, Nip66 | None, Exception | None]:
    relay = Relay(url)
    try:
        result = await Nip66.test(
            relay,
            timeout=15.0,
            keys=keys,
            event_builder=EventBuilder.text_note(f"Test {url}"),
            read_filter=Filter().limit(1),
            city_reader=city_reader,
            asn_reader=asn_reader,
        )
        return url, result, None
    except Exception as e:
        return url, None, e

results = await asyncio.gather(*[test_nip66_safe(url) for url in relay_urls])

print("=== Results ===")
for url, result, error in results:
    if result:
        rtt = result.rtt_metadata.data if result.rtt_metadata else {}
        geo = result.geo_metadata.data if result.geo_metadata else {}
        print(f"\n{url}")
        print(f"  RTT: {rtt.get('rtt_open', 'N/A')}ms | {geo.get('geo_country', '?')} | {geo.get('geo_asn_org', 'Unknown')}")
    else:
        print(f"\n{url}: ERROR - {type(error).__name__}")

22:00:39 INFO  models.nip66 | test: relay=wss://relay.damus.io timeout=15.0s
22:00:39 DEBUG models.nip66 | test: running tests=['dns', 'rtt', 'ssl', 'geo', 'http']
22:00:39 INFO  models.nip66 | test: relay=wss://nos.lol timeout=15.0s
22:00:39 DEBUG models.nip66 | test: running tests=['dns', 'rtt', 'ssl', 'geo', 'http']
22:00:39 INFO  models.nip66 | test: relay=wss://relay.nostr.band timeout=15.0s
22:00:39 DEBUG models.nip66 | test: running tests=['dns', 'rtt', 'ssl', 'geo', 'http']
22:00:39 INFO  models.nip66 | test: relay=wss://nostr.wine timeout=15.0s
22:00:39 DEBUG models.nip66 | test: running tests=['dns', 'rtt', 'ssl', 'geo', 'http']
22:00:39 DEBUG models.nip66 | _test_dns: relay=wss://relay.damus.io timeout=15.0s
22:00:39 DEBUG models.nip66 | _test_dns: resolving host=relay.damus.io
22:00:39 DEBUG models.nip66 | _test_rtt: relay=wss://relay.damus.io timeout=15.0s proxy=None
22:00:39 DEBUG models.nip66 | _test_ssl: relay=wss://relay.damus.io timeout=15.0s
22:00:39 DEBUG models.nip

=== Results ===

wss://relay.damus.io
  RTT: 509ms | US | CLOUDFLARENET

wss://nos.lol
  RTT: 176ms | DE | Hetzner Online GmbH

wss://relay.nostr.band
  RTT: N/Ams | FI | Hetzner Online GmbH

wss://nostr.wine
  RTT: 459ms | US | CLOUDFLARENET


## 11. Test with Proxy (Tor Relay)

**Important**: Only `_test_rtt` and `_test_http` use the proxy. Other tests (SSL, DNS, GEO) are clearnet-only.

To test Tor relays, you need a running Tor proxy (SOCKS5 on port 9050).

In [14]:
# Example: Test a Tor relay (requires Tor proxy running)
# Uncomment to test if you have Tor running locally

# tor_relay = Relay("ws://oxtrdevav64z64yb7x6rjg3a4a7kblqcmjzreo5hsktyqhmpxsylzead.onion")
# tor_proxy = "socks5://127.0.0.1:9050"
# 
# nip66_tor = await Nip66.test(
#     tor_relay,
#     timeout=30.0,
#     keys=keys,
#     event_builder=event_builder,
#     read_filter=read_filter,
#     proxy_url=tor_proxy,  # Required for overlay networks
# )
# 
# # For Tor relay: RTT and HTTP work via proxy, SSL/DNS/GEO return empty
# print(f"RTT: {nip66_tor.rtt_metadata is not None}")  # True (via proxy)
# print(f"HTTP: {nip66_tor.http_metadata is not None}")  # True (via proxy)
# print(f"SSL: {nip66_tor.ssl_metadata is not None}")   # False (clearnet only)
# print(f"DNS: {nip66_tor.dns_metadata is not None}")   # False (clearnet only)
# print(f"GEO: {nip66_tor.geo_metadata is not None}")   # False (clearnet only)

print("Tor test skipped (uncomment to run with local Tor proxy)")

Tor test skipped (uncomment to run with local Tor proxy)


## 12. Proxy Behavior by Test Type

| Test | Uses Proxy | Clearnet | Overlay (with proxy) | Overlay (no proxy) |
|------|------------|----------|---------------------|--------------------|
| RTT  | Yes        | Works    | Works               | Fails              |
| HTTP | Yes        | Works    | Works               | Skipped (empty)    |
| SSL  | No         | Works    | Skipped (empty)     | Skipped (empty)    |
| DNS  | No         | Works    | Skipped (empty)     | Skipped (empty)    |
| GEO  | No         | Works    | Skipped (empty)     | Skipped (empty)    |

## 13. Selective Tests (run_* flags)

In [15]:
# Test DNS only
nip66_dns_only = await Nip66.test(
    relay,
    run_rtt=False,
    run_ssl=False,
    run_geo=False,
    run_http=False,
)

print("=== DNS Only ===")
print(f"dns_metadata: {nip66_dns_only.dns_metadata is not None}")
if nip66_dns_only.dns_metadata:
    print(f"DNS IP: {nip66_dns_only.dns_metadata.data['dns_ip']}")

22:00:54 INFO  models.nip66 | test: relay=wss://relay.damus.io timeout=10.0s
22:00:54 DEBUG models.nip66 | test: running tests=['dns']
22:00:54 DEBUG models.nip66 | _test_dns: relay=wss://relay.damus.io timeout=10.0s
22:00:54 DEBUG models.nip66 | _test_dns: resolving host=relay.damus.io
22:01:05 DEBUG models.nip66 | _test_dns: completed relay=wss://relay.damus.io ip=172.67.206.128 rtt=64ms
22:01:05 DEBUG models.nip66 | test: dns succeeded
22:01:05 INFO  models.nip66 | test: completed relay=wss://relay.damus.io


=== DNS Only ===
dns_metadata: True
DNS IP: 172.67.206.128


In [16]:
# Test SSL only
nip66_ssl_only = await Nip66.test(
    relay,
    run_rtt=False,
    run_geo=False,
    run_dns=False,
    run_http=False,
)

print("=== SSL Only ===")
print(f"ssl_metadata: {nip66_ssl_only.ssl_metadata is not None}")
if nip66_ssl_only.ssl_metadata:
    print(f"Protocol: {nip66_ssl_only.ssl_metadata.data['ssl_protocol']}")

22:01:05 INFO  models.nip66 | test: relay=wss://relay.damus.io timeout=10.0s
22:01:05 DEBUG models.nip66 | test: running tests=['ssl']
22:01:05 DEBUG models.nip66 | _test_ssl: relay=wss://relay.damus.io timeout=10.0s
22:01:05 DEBUG models.nip66 | _test_ssl: checking host=relay.damus.io port=443
22:01:05 DEBUG models.nip66 | _test_ssl: completed relay=wss://relay.damus.io valid=True
22:01:05 DEBUG models.nip66 | test: ssl succeeded
22:01:05 INFO  models.nip66 | test: completed relay=wss://relay.damus.io


=== SSL Only ===
ssl_metadata: True
Protocol: TLSv1.3


## 14. Error Handling

In [17]:
# Test: Missing required parameters for RTT
try:
    result = await Nip66.test(
        relay, 
        run_rtt=True,
        run_ssl=False,
        run_geo=False,
        run_dns=False,
        run_http=False,
    )  # Missing keys, event_builder, and read_filter!
except Nip66TestError as e:
    print("=== Missing Parameters ===")
    print(f"Correctly raised: {e}")

22:01:05 INFO  models.nip66 | test: relay=wss://relay.damus.io timeout=10.0s
22:01:05 DEBUG models.nip66 | test: running tests=['rtt']
22:01:05 DEBUG models.nip66 | _test_rtt: relay=wss://relay.damus.io timeout=10.0s proxy=None
22:01:05 WARNING models.nip66 | _test_rtt: missing required params relay=wss://relay.damus.io
22:01:05 DEBUG models.nip66 | test: rtt failed: Failed to test NIP-66 for wss://relay.damus.io: RTT test requires keys, event_builder and read_filter
22:01:05 WARNING models.nip66 | test: all tests failed relay=wss://relay.damus.io


=== Missing Parameters ===
Correctly raised: Failed to test NIP-66 for wss://relay.damus.io: At least one NIP-66 metadata must have data


In [18]:
# Test: Empty metadata raises error
try:
    nip66_empty = Nip66(
        relay=relay,
        rtt_metadata=Metadata({}),
        ssl_metadata=Metadata({}),
        geo_metadata=Metadata({}),
        dns_metadata=Metadata({}),
        http_metadata=Metadata({}),
    )
except ValueError as e:
    print("=== Empty Metadata ===")
    print(f"Correctly raised: {e}")

=== Empty Metadata ===
Correctly raised: At least one NIP-66 metadata must have data


## 15. Class Defaults

In [19]:
print("=== Class Defaults ===")
print(f"_DEFAULT_TEST_TIMEOUT: {Nip66._DEFAULT_TEST_TIMEOUT} seconds")

=== Class Defaults ===
_DEFAULT_TEST_TIMEOUT: 10.0 seconds


## 16. Cleanup

## Done!

### Key Points:
- Access fields via `nip66.<type>_metadata.data["key"]`
- All schema keys are present (with `None` for missing)
- Invalid types are silently converted to `None`
- At least one metadata type must have data
- **RTT test requires:** `keys`, `event_builder`, `read_filter`
- **Only RTT and HTTP tests use `proxy_url`**
- SSL, DNS, GEO are clearnet-only (no proxy support)

### Metadata Types

| Type | Fields | Proxy |
|------|--------|-------|
| RTT  | rtt_open, rtt_read, rtt_write | Yes |
| HTTP | http_server, http_powered_by | Yes |
| SSL  | ssl_valid, ssl_issuer, ssl_protocol, ... | No |
| DNS  | dns_ip, dns_ipv6, dns_ns, dns_ttl, ... | No |
| GEO  | geo_country, geo_city, geo_asn, ... | No |

In [23]:
relay = Relay("wss://relay.nostr.band")
relay = Relay("wss://nos.lol")
# relay = Relay("wss://relay.damus.io")
# relay = Relay("ws://oxtrdevav64z64yb7x6rjg4ntzqjhedm5b5zjqulugknhzr46ny2qbad.onion")
# relay = Relay("wss://sdsdsdsddsdsdfhffrfghjrj.io")
print(f"Testing: {relay.url}")
print(f"Network: {relay.network}")

event = EventBuilder.text_note("NIP-66 test event")
read_filter = Filter().limit(1)

result = await Nip66.test(
    relay, 
    event_builder=event, 
    keys=keys, 
    read_filter=read_filter,
    timeout=10.0, 
    city_reader=city_reader, 
    asn_reader=asn_reader
)

print(f"\nMetadata collected:")
print(f"  RTT:  {result.rtt_metadata is not None} {result.rtt_metadata.data if result.rtt_metadata else ''}")
print(f"  SSL:  {result.ssl_metadata is not None} {result.ssl_metadata.data if result.ssl_metadata else ''}")
print(f"  GEO:  {result.geo_metadata is not None} {result.geo_metadata.data if result.geo_metadata else ''}")
print(f"  DNS:  {result.dns_metadata is not None} {result.dns_metadata.data if result.dns_metadata else ''}")
print(f"  HTTP: {result.http_metadata is not None} {result.http_metadata.data if result.http_metadata else ''}")

22:02:37 INFO  models.nip66 | test: relay=wss://nos.lol timeout=10.0s
22:02:37 DEBUG models.nip66 | test: running tests=['dns', 'rtt', 'ssl', 'geo', 'http']
22:02:37 DEBUG models.nip66 | _test_dns: relay=wss://nos.lol timeout=10.0s
22:02:37 DEBUG models.nip66 | _test_dns: resolving host=nos.lol
22:02:37 DEBUG models.nip66 | _test_rtt: relay=wss://nos.lol timeout=10.0s proxy=None
22:02:37 DEBUG models.nip66 | _test_ssl: relay=wss://nos.lol timeout=10.0s
22:02:37 DEBUG models.nip66 | _test_ssl: checking host=nos.lol port=443
22:02:37 DEBUG models.nip66 | _test_geo: relay=wss://nos.lol city=True asn=True
22:02:37 DEBUG models.nip66 | _test_geo: resolving host=nos.lol
22:02:37 DEBUG models.nip66 | _test_http: relay=wss://nos.lol timeout=10.0s proxy=None
22:02:37 DEBUG models.nip66 | _test_rtt: connecting relay=wss://nos.lol
22:02:37 DEBUG models.nip66 | _test_geo: resolved ip=5.9.78.12 relay=wss://nos.lol
22:02:37 DEBUG models.nip66 | _test_geo: completed relay=wss://nos.lol country=DE
22:

Testing: wss://nos.lol
Network: clearnet


22:02:37 DEBUG models.nip66 | _test_rtt: open succeeded relay=wss://nos.lol rtt_open=199ms
22:02:37 DEBUG models.nip66 | _test_rtt: reading relay=wss://nos.lol
22:02:37 DEBUG models.nip66 | _test_rtt: read event relay=wss://nos.lol rtt_read=84ms
22:02:37 DEBUG models.nip66 | _test_rtt: writing relay=wss://nos.lol
22:02:37 DEBUG models.nip66 | _test_rtt: write accepted relay=wss://nos.lol
22:02:37 DEBUG models.nip66 | _test_rtt: write failed relay=wss://nos.lol error='EventId' object is not callable
22:02:37 DEBUG models.nip66 | _test_rtt: completed relay=wss://nos.lol data=['rtt_open', 'rtt_read']
22:02:37 DEBUG models.nip66 | test: dns succeeded
22:02:37 DEBUG models.nip66 | test: rtt succeeded
22:02:37 DEBUG models.nip66 | test: ssl succeeded
22:02:37 DEBUG models.nip66 | test: geo succeeded
22:02:37 DEBUG models.nip66 | test: http succeeded
22:02:37 INFO  models.nip66 | test: completed relay=wss://nos.lol



Metadata collected:
  RTT:  True {'rtt_open': 199, 'rtt_read': 84, 'rtt_write': None}
  SSL:  True {'ssl_valid': True, 'ssl_subject_cn': None, 'ssl_issuer': None, 'ssl_issuer_cn': None, 'ssl_expires': None, 'ssl_not_before': None, 'ssl_san': None, 'ssl_serial': None, 'ssl_version': None, 'ssl_fingerprint': 'SHA256:61:9F:78:1F:F2:65:8E:10:3A:29:72:CE:F1:96:A8:97:28:1A:7A:E4:C4:99:44:2B:D2:AE:4C:D5:50:87:81:F3', 'ssl_protocol': 'TLSv1.3', 'ssl_cipher': 'TLS_AES_256_GCM_SHA384', 'ssl_cipher_bits': 256}
  GEO:  True {'geo_ip': '5.9.78.12', 'geo_country': 'DE', 'geo_country_name': 'Germany', 'geo_continent': 'EU', 'geo_continent_name': 'Europe', 'geo_is_eu': True, 'geo_region': 'Saxony', 'geo_city': 'Falkenstein', 'geo_postal': '08223', 'geo_lat': 50.4777, 'geo_lon': 12.3649, 'geo_accuracy': 20, 'geo_tz': 'Europe/Berlin', 'geohash': 'u2bz1m5vg', 'geo_geoname_id': 2927913, 'geo_asn': 24940, 'geo_asn_org': 'Hetzner Online GmbH', 'geo_network': '5.9.0.0/16'}
  DNS:  True {'dns_ip': '5.9.78.12